In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
import os
import numpy as np
import cv2

In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, Dense, BatchNormalization
from keras.optimizers import Adam

In [ ]:
os.chdir('/content/gdrive/MyDrive/AI in Healthcare/train')

In [ ]:
def preprocess(path):
  cap = cv2.imread(path)
  cap = cv2.cvtColor(cap, cv2.COLOR_RGB2GRAY)
  cap = cv2.resize(cap, (64, 64), interpolation=cv2.INTER_AREA)
  return cap

In [ ]:
classes = []
images = []
for i in os.listdir():
  for j in os.listdir(i):
    image = preprocess(i+'/'+j)
    images.append(image)
    if i == 'normal':
      classes.append(0)
    else:
      classes.append(1)

In [ ]:
pred = []
pred.append(preprocess('pneumonia/CXR_train_1109.png'))
pred = np.array(pred)
pred = pred/255
pred = pred.reshape((-1,32,32,1))

In [ ]:
images = np.array(images)
images = images/255
classes = np.array(classes)
images = images.reshape((-1,64,64,1))

In [ ]:
images.shape

(2122, 64, 64, 1)

In [ ]:
def mymod():
    model = Sequential()
    model.add(Conv2D(128, (5,5), input_shape=(64,64,1), activation='relu'))
    model.add(Conv2D(64, (5,5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # model.add(Conv2D(112, (11,11), activation='relu'))
    # model.add(Conv2D(112, (11,11), activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))

    # model.add(Conv2D(64, (5,5), activation='relu'))
    # model.add(Conv2D(64, (5,5), activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(Conv2D(16, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dense(500, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    model.compile(Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = mymod()
print(model.summary())

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_88 (Conv2D)          (None, 60, 60, 128)       3328      
                                                                 
 conv2d_89 (Conv2D)          (None, 56, 56, 64)        204864    
                                                                 
 max_pooling2d_49 (MaxPoolin  (None, 28, 28, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_90 (Conv2D)          (None, 26, 26, 32)        18464     
                                                                 
 conv2d_91 (Conv2D)          (None, 24, 24, 16)        4624      
                                                                 
 max_pooling2d_50 (MaxPoolin  (None, 12, 12, 16)       0         
 g2D)                                                

In [ ]:
model.fit(images,classes,batch_size=32, epochs=10)

Epoch 1/10
67/67 [==============================] - 2s 25ms/step - loss: 0.5901 - accuracy: 0.6979
Epoch 2/10
67/67 [==============================] - 1s 20ms/step - loss: 0.5879 - accuracy: 0.6975
Epoch 3/10
67/67 [==============================] - 1s 20ms/step - loss: 0.5748 - accuracy: 0.7069
Epoch 4/10
67/67 [==============================] - 1s 20ms/step - loss: 0.5646 - accuracy: 0.7116
Epoch 5/10
67/67 [==============================] - 1s 20ms/step - loss: 0.5626 - accuracy: 0.7059
Epoch 6/10
67/67 [==============================] - 1s 20ms/step - loss: 0.5609 - accuracy: 0.7144
Epoch 7/10
67/67 [==============================] - 1s 20ms/step - loss: 0.5506 - accuracy: 0.7253
Epoch 8/10
67/67 [==============================] - 1s 20ms/step - loss: 0.5459 - accuracy: 0.7253
Epoch 9/10
67/67 [==============================] - 1s 20ms/step - loss: 0.5354 - accuracy: 0.7257
Epoch 10/10
67/67 [==============================] - 1s 20ms/step - loss: 0.5151 - accuracy: 0.7460


In [ ]:
print(np.argmax(model.predict(pred), axis=1))

1/1 [==============================] - 0s 144ms/step
[1]


In [ ]:
os.chdir('/content/gdrive/MyDrive/Colab Notebooks')

In [ ]:
import pickle
pickle_in = open('kbgmodel.p','rb')
model = pickle.load(pickle_in)

In [ ]:
os.chdir('/content/gdrive/MyDrive/AI in Healthcare/test/AIIHCT')

In [ ]:
images_test = []
img = []
for j in os.listdir():
  image = preprocess(j)
  images_test.append(image)

In [ ]:
images_test = np.array(images_test)
images_test = images_test/255
images_test = images_test.reshape((-1,64,64,1))

In [ ]:
pred_test = np.argmax(model.predict(images_test), axis=1)

10/10 [==============================] - 0s 8ms/step


In [ ]:
err = 0
for i in (np.argmax(model.predict(images_test), axis=1)-pred_test):
  if i!=0:
    
    err+=1
err

10/10 [==============================] - 0s 3ms/step


72

In [ ]:
print(pred_test)

[0 0 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 0 1 0 1 1 0 0 0
 1 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0
 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0 1
 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 0 1 1 1 1 0 0 1 1 1 0
 0 0 0 0 1 1 0 0 0 0 1 0 0 1 1 1 0 0 1 0 1 1 0 1 1 1 1 1 0 0 1 0 1 1 1 0 0
 1 1 0 1 1 1 0 1 1 0 1 1 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0
 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 1 0 0 1 1 1 1 1 1 1 1
 1 0 1 1 1 1 0 1 0 0 1 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0
 1 0 0 0]


In [ ]:
print(pred_test)

[0 0 0 0 0 1 1 0 0 1 0 1 1 0 0 1 0 1 1 0 1 0 1 1 0 1 0 0 1 0 1 0 1 1 0 0 0
 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 0 1
 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0
 0 0 0 0 1 1 0 0 0 0 1 0 0 1 1 1 1 0 1 0 1 1 1 1 1 1 0 0 0 0 1 0 1 1 1 0 1
 1 1 0 1 1 0 0 1 0 0 1 1 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 1 0 1 1 1 1 0 0 1 0
 0 0 0 1 1 0 0 1 1 1 1 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 1 1 1 0 1
 1 0 1 0 1 0 0 1 0 0 0 1 0 1 0 1 0 1 0 1 1 0 0 1 0 1 0 0 0 0 1 1 0 0 0 0 0
 1 0 0 0]


In [ ]:
print(pred_test)

[0 1 0 1 1 1 1 1 1 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 1 0 1 1 1 0 0
 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 1 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 1 0
 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 1 0 1
 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 1 1 1 1 0 0 1 0 0 0
 0 0 1 0 1 1 0 0 0 0 1 1 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 0 1 0 1 1 1 0 1
 1 1 0 1 1 1 0 0 1 0 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 0
 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 1 0 1 0 0 1 0 0 1 0 1 1 1
 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0
 1 0 1 1]


In [5]:
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/KBG')

In [3]:
import pandas as pd

In [17]:
import pandas as pd
pred_dict = {'label':fin}
df = pd.DataFrame(pred_dict)
df.to_csv('phyve_final2.csv', index=False)

In [7]:
pre1 = pd.read_csv('phyve_1.csv')
pre2 = pd.read_csv('phyve_2.csv')
pre3 = pd.read_csv('phyve_final.csv')
pre4 = pd.read_csv('phyve_4.csv')
pre5 = pd.read_csv('phyve_5.csv')

In [ ]:
pre6 = pd.read_csv('phyve_6.csv')

In [9]:
final = []
lis = list(zip(pre1.label,pre2.label,pre3.label,pre4.label,pre5.label))
for i in range(300):
  if lis[i] != (0,0,0,0,0) and lis[i] != (1,1,1,1,1):
    final.append(i)
print(final)
print(len(final))

[1, 3, 4, 6, 7, 8, 9, 10, 11, 12, 18, 19, 20, 21, 23, 24, 25, 31, 33, 34, 37, 40, 42, 44, 48, 49, 50, 54, 55, 56, 58, 59, 64, 66, 68, 72, 78, 82, 88, 91, 95, 97, 98, 99, 100, 101, 103, 105, 106, 108, 110, 114, 116, 117, 118, 120, 125, 126, 127, 128, 130, 132, 133, 134, 135, 137, 140, 141, 146, 150, 158, 159, 160, 161, 164, 165, 166, 170, 173, 174, 175, 178, 180, 184, 185, 190, 192, 193, 195, 198, 200, 201, 206, 207, 214, 215, 216, 218, 222, 223, 225, 226, 230, 231, 232, 235, 236, 237, 238, 239, 240, 241, 242, 244, 245, 246, 247, 248, 249, 250, 252, 253, 254, 255, 257, 259, 261, 262, 263, 264, 265, 269, 270, 272, 275, 276, 278, 279, 281, 284, 286, 289, 292, 296, 298, 299]
146


In [11]:
fin = pre3.label.tolist()

In [12]:
for i in final:
  if fin[i] == 0:
    fin[i] = 1
  else:
    fin[i] = 0

In [13]:
sum = 0
for i in range(300):
  if pre3.label.tolist()[i] != fin[i]:
    sum+=1

In [14]:
sum

146

In [ ]:
os.chdir('/content/gdrive/MyDrive/AI in Healthcare/train/normal')

In [ ]:
images_norm = []
for j in os.listdir():
  image = preprocess(j)
  images_norm.append(image)
images_norm = np.array(images_norm)
images_norm = images_norm/255
images_norm = images_norm.reshape((-1,32,32,1))

In [ ]:
pred_norm = np.argmax(model.predict(images_norm), axis=1)

36/36 [==============================] - 0s 5ms/step


In [ ]:
s = 0
for i in pred_norm:
  s+=i

In [16]:
list(zip(fin, pre3.label.tolist()))

[(0, 0),
 (0, 1),
 (0, 0),
 (0, 1),
 (0, 1),
 (1, 1),
 (0, 1),
 (1, 0),
 (1, 0),
 (1, 0),
 (0, 1),
 (1, 0),
 (0, 1),
 (0, 0),
 (0, 0),
 (1, 1),
 (0, 0),
 (1, 1),
 (1, 0),
 (0, 1),
 (1, 0),
 (0, 1),
 (1, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 0),
 (0, 0),
 (1, 1),
 (0, 0),
 (1, 1),
 (0, 1),
 (1, 1),
 (1, 0),
 (0, 1),
 (0, 0),
 (0, 0),
 (0, 1),
 (0, 0),
 (0, 0),
 (0, 1),
 (1, 1),
 (0, 1),
 (0, 0),
 (0, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 0),
 (0, 1),
 (1, 0),
 (1, 1),
 (1, 1),
 (1, 1),
 (0, 1),
 (1, 0),
 (0, 1),
 (0, 0),
 (0, 1),
 (0, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 1),
 (0, 0),
 (1, 0),
 (1, 1),
 (0, 1),
 (1, 1),
 (0, 0),
 (0, 0),
 (0, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 1),
 (1, 1),
 (0, 0),
 (0, 0),
 (0, 1),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 1),
 (1, 1),
 (1, 1),
 (1, 0),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 0),
 (0, 0),
 (1, 0),
 (0, 1),
 (1, 0),
 (0, 1),
 (0, 1),
 (0, 0),
 (0, 1),
 (0, 0),
 (1, 0),
 (1, 0),
 (0, 0),
 (1, 0),
 (0, 0),
 (0, 1),
 